# MACHINE LEARNING 

- Data Cleaning and Preparation
- Model Preparation
- Elastic Net Prediction Model
- 10-Fold Cross Validation

<a href= 'https://github.com/YuvalBaron1997/ML---cars/blob/main/ML.ipynb'>Click here to visit my GitHub</a>

Yuval Bar-On

In [55]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import math
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.model_selection import cross_val_score





In [30]:
url = 'https://raw.githubusercontent.com/YuvalBaron1997/ML---cars/main/dataset.csv'

sr_df = pd.read_csv(url, encoding='utf-8')
data_frame = sr_df.copy()
data_frame.head()

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,יונדאי,2015,i35,2,אוטומטית,1600,בנזין,פרטית,פרטית,רעננה - כפר סבא,רעננה,51000.0,2.0,11/07/2023,11/07/2023,['רכב שמור בקנאות\nמוכרת עקב קבלת רכב חברה'],כחול כהה מטאלי,144000,NaN,NaN
1,ניסאן,2018,ניסאן מיקרה,1,אוטומטית,1200,בנזין,פרטית,פרטית,מושבים בשרון,אבן יהודה,49000.0,0.0,06/04/2022,22/05/2022,['שמורה כל התוספות'],כחול בהיר,69000,NaN,NaN
2,סוזוקי,2010,סוזוקי סוויפט,1,אוטומטית,1450,בנזין,NaN,NaN,רמת,רמת,22500.0,1.0,29/10/2022,29/10/2022,['רכב במצב מתוחזק ברמה גבוהה טסט עד אפריל 2023'],None,145000,NaN,NaN
3,טויוטה,2016,אוריס,1,טיפטרוניק,1600,בנזין,פרטית,פרטית,נס ציונה - רחובות,רחובות,63000.0,5.0,16/05/2024,16/05/2024,['אוטו במצב חדש!! שמור בקנאות!! נהג יחיד מטופל...,אפור מטאלי,27300,NaN,NaN
4,קיה,2012,פיקנטו,1,אוטומטית,1248,בנזין,NaN,NaN,"ראשל""צ והסביבה",ראשון לציון,37000.0,1.0,13/06/2022,13/06/2022,['שמור'],NaN,70000,NaN,4.0


In [40]:
print(data_frame.info())
#print(data_frame.count())
data_frame['Engine_type'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   manufactor       1500 non-null   object 
 1   Year             1500 non-null   int64  
 2   model            1500 non-null   object 
 3   Hand             1500 non-null   int64  
 4   Gear             1499 non-null   object 
 5   capacity_Engine  1474 non-null   object 
 6   Engine_type      1495 non-null   object 
 7   Prev_ownership   774 non-null    object 
 8   Curr_ownership   774 non-null    object 
 9   Area             1361 non-null   object 
 10  City             1500 non-null   object 
 11  Price            1500 non-null   float64
 12  Pic_num          1476 non-null   float64
 13  Cre_date         1500 non-null   object 
 14  Repub_date       1500 non-null   object 
 15  Description      1500 non-null   object 
 16  Color            787 non-null    object 
 17  Km            

array(['בנזין', 'דיזל', 'גז', 'היברידי', 'היבריד', 'טורבו דיזל', nan,
       'חשמלי'], dtype=object)

* Data Cleaning and Preparation

In [70]:

def prepare_data(data):
    # סידור - הורדת העמודות עם יותר מחצי ערכים חסרים
    data = data.drop(columns=['Test', 'Supply_score'])
    
    # רשימת הערכים להחלפה
    values_to_replace = ['None', 'nan', 'לא מוגדר']

    # מעבר על כל התאים בכל העמודות והחלפת הערכים
    for col in data.columns:
        data[col] = data[col].apply(lambda x: None if x in values_to_replace else x)
    data = data.applymap(lambda x: np.nan if pd.isna(x) else x)
    
    ######################################################################################################################
    # סידור עמודות פר עמודה 
    
    # manufactor - במידה ויהיו ערכים חסרים נשלים אותם על ידי זיהוי המודל.
    data.loc[:, 'manufactor'] = data['manufactor'].replace('Lexsus', 'לקסוס')
    data['manufactor'] = data.groupby('model')['manufactor'].transform(lambda x: x.fillna(x.mode()[0]))
    
    # Gear - בקירוב ל-2005 התחילו לייצר מכוניות אוטומטיות במקום עם הילוכים. לכן אם יהיה לנו ערך חסר זה ילך על פי העובדה הזאת.
    data.loc[:, 'Gear'] = data['Gear'].replace('אוטומט', 'אוטומטית')
    data['Gear'] = data.apply(lambda row: 'אוטומטית' if pd.isna(row['Gear']) and row['Year'] >= 2005 else 
                    'ידנית' if pd.isna(row['Gear']) and row['Year'] < 2005 else row['Gear'], axis=1)
    
    # capacity_Engine - הפיכת ערכים למספר + אם זה קטן מ-400 ערך הכי קטן למנוע אז להכפיל ב100 + מילוי על ידי חציון
    for idx in data.index:
        value = data.loc[idx, 'capacity_Engine']
        if isinstance(value, str):
            clean_value = value.replace(',', '')
            data.loc[idx, 'capacity_Engine'] = int(clean_value)
    
    # הכפלת ערכים קטנים מ-400 ב-100
    for idx in data.index:
        value = data.loc[idx, 'capacity_Engine']
        if value < 400:
            data.loc[idx, 'capacity_Engine'] = value * 100

    # חישוב החציוני של העמודה ומילוי הערכים החסרים
    median_value = data['capacity_Engine'].median()
    data['capacity_Engine'].fillna(median_value, inplace=True)
    
    # 'City' ו-'Area' - מילוי איזור על ידי העיר + הפיכה למספרים קטגוריאלים
    data['Area'] = data.groupby('City')['Area'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)
    data['City'] = data['City'].astype(str)
    data['Area'] = data['Area'].astype(str)
    
    data['City'] = data['City'].str.strip()
    data['Area'] = data['Area'].str.strip()
    
    data['City'] = pd.Categorical(data['City']).codes
    data['Area'] = pd.Categorical(data['Area']).codes
    
    # סידור עמודת 'Engine_type'
    data.loc[:, 'Engine_type'] = data['Engine_type'].replace('היבריד', 'היברידי')
    
    # מילוי ערכים חסרים בעמודת Engine_type לפי ערך שכיח
    if 'Engine_type' in data.columns:
        most_common_engine_type = data['Engine_type'].mode()[0] if not data['Engine_type'].mode().empty else np.nan
        data['Engine_type'].fillna(most_common_engine_type, inplace=True)
    
    # מניפולציות על עמודת Km
    for idx in data.index:
        value = data.loc[idx, 'Km']
        if isinstance(value, str):
            # הסרת פסיקים והמרה למספר שלם
            clean_value = value.replace(',', '')
            data.loc[idx, 'Km'] = int(clean_value)
        
        # הכפלת ערכים קטנים מ-1000 ב-1000
        if isinstance(data.loc[idx, 'Km'], int) and data.loc[idx, 'Km'] < 1000:
            data.loc[idx, 'Km'] *= 1000
    
    # טיפול בערכים חסרים בעמודת Km על פי הנוסחה
    current_year = 2024
    data['Km'] = data.apply(
        lambda row: (current_year - row['Year']) * 12000 if pd.isna(row['Km']) and pd.notna(row['Year']) else row['Km'],
        axis=1)
    #print(data['Km'].unique())
    
    # Color - הסרת מילים שלא תורמות + השלמת ערכים על ידי ערך שכיח
    if 'Color' in data.columns:
        words_to_remove = ["פנינה","בהיר", "כהה", "מטאלי", "מטלי"]
        for idx in data.index:
            value = data.loc[idx, 'Color']
            if isinstance(value, str):
                words = value.split()
                filtered_words = [word for word in words if word not in words_to_remove]
                data.loc[idx, 'Color'] = ' '.join(filtered_words)
    
    # מציאת הצבע השכיח ביותר
    most_common_color = data['Color'].mode()[0] if not data['Color'].mode().empty else np.nan
    
    # מילוי ערכים חסרים בצבע השכיח ביותר
    data['Color'].fillna(most_common_color, inplace=True)
    
    # מילוי ערכים חסרים בעמודות Prev_ownership ו-Curr_ownership
    if 'Prev_ownership' in data.columns:
        data['Prev_ownership'].fillna('אחר', inplace=True)
    
    if 'Curr_ownership' in data.columns:
        data['Curr_ownership'].fillna('אחר', inplace=True)
    
    # הסרת העמודות המיותרות
    columns_to_drop = ['model', 'City', 'Pic_num', 'Cre_date', 'Repub_date', 'Description']
    data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])
    
    return data

# הפעלת הפונקציה
prepared_df = prepare_data(data_frame)
prepared_df

,manufactor,Year,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,Price,Color,Km
0,יונדאי,2015,2,אוטומטית,1600.0,בנזין,פרטית,פרטית,56,51000.0,כחול,144000
1,ניסאן,2018,1,אוטומטית,1200.0,בנזין,פרטית,פרטית,31,49000.0,כחול,69000
2,סוזוקי,2010,1,אוטומטית,1450.0,בנזין,אחר,אחר,52,22500.0,לבן,145000
3,טויוטה,2016,1,טיפטרוניק,1600.0,בנזין,פרטית,פרטית,33,63000.0,אפור,27300
4,קיה,2012,1,אוטומטית,1248.0,בנזין,אחר,אחר,48,37000.0,לבן,70000
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,סקודה,2016,2,אוטומטית,1400.0,בנזין,אחר,אחר,46,60000.0,לבן,180000
1496,אלפא רומיאו,2013,4,ידנית,1400.0,בנזין,אחר,אחר,17,45000.0,לבן,160000
1497,סקודה,2014,1,אוטומטית,1400.0,בנזין,אחר,אחר,0,30000.0,לבן,120000
1498,ניסאן,2011,3,אוטומטית,1600.0,בנזין,פרטית,פרטית,3,28000.0,אפור,118000


* Model Preparation 


In [45]:
###################################################################################################################
# one hot encoder
select_columns_1_hot = ['manufactor', 'Gear', 'Engine_type', 'Area', 'Color']

data_hot = prepared_df[select_columns_1_hot]

one_hot_encoder = OneHotEncoder()

# create a copy of the dataset
df_ohe = data_hot.copy()

# fit one hot encoder
one_hot_encoder = one_hot_encoder.fit(df_ohe)

# transform dataset 
ohelabels = one_hot_encoder.transform(df_ohe).toarray()
df_ohe = pd.DataFrame(ohelabels, columns=one_hot_encoder.get_feature_names())

###################################################################################################################
# Standardization
select_columns_Standardization = ['Year', 'capacity_Engine', 'Km', 'Hand']

data_stand = prepared_df[select_columns_Standardization]
scaler = preprocessing.StandardScaler()
scaler.fit(data_stand)
df_le_arr = scaler.transform(data_stand)
data_s = pd.DataFrame(df_le_arr, columns=select_columns_Standardization)

#####################################################################################################################
# חיבור עמודות - הדאטה הסופית לפונקציה
df_ohe['Year'] = data_s['Year']
df_ohe['capacity_Engine'] = data_s['capacity_Engine']
df_ohe['Km'] = data_s['Km']
df_ohe['Price'] = prepared_df['Price']

df_ohe.head()


,x0_אאודי,x0_אופל,x0_אלפא רומיאו,x0_ב.מ.וו,x0_דייהטסו,x0_הונדה,x0_וולוו,x0_טויוטה,x0_יונדאי,x0_לקסוס,...,x4_לבן שנהב,x4_סגול,x4_סגול חציל,x4_שחור,x4_שמפניה,x4_תכלת,Year,capacity_Engine,Km,Price
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.171467,-0.111086,0.143930,51000.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.958016,-0.491382,-0.783344,49000.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.139446,-0.253697,0.156294,22500.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.433650,-0.111086,-1.298908,63000.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.615081,-0.445746,-0.770980,37000.0


* Elastic Net Prediction Model

In [69]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
import math

# הגדרת המשתנים המסבירים והמשתנה המוסבר
X = df_ohe.drop('Price', axis=1)
y = df_ohe['Price']

# חלוקת הדאטה ל-90% train ו-10% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# הדפסה כדי לראות את אורך כל קבוצה
print(f'Length of X_train: {len(X_train)}')
print(f'Length of X_test: {len(X_test)}')
print(f'Length of y_train: {len(y_train)}')
print(f'Length of y_test: {len(y_test)}')

# יצירת מודל ElasticNet
Elastic = ElasticNet()

# הגדרת הפרמטרים לחיפוש
param_grid = {
    'alpha': [0.1, 1, 10, 100],
    'l1_ratio': [0.1, 0.5, 0.9]
}

# יצירת חיפוש גריד לחיפוש האלפא וה-l1_ratio הטובים ביותר
grid_search = GridSearchCV(estimator=Elastic, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# אימון החיפוש גריד
grid_search.fit(X_train, y_train)

# הדפסת הפרמטרים הטובים ביותר
print(f'Best alpha: {grid_search.best_params_["alpha"]}')
print(f'Best l1_ratio: {grid_search.best_params_["l1_ratio"]}')

# יצירת מודל ElasticNet עם הפרמטרים הטובים ביותר
best_Elastic = ElasticNet(alpha=grid_search.best_params_['alpha'], l1_ratio=grid_search.best_params_['l1_ratio'])

# אימון המודל
best_Elastic.fit(X_train, y_train)

# ביצוע חיזוי
pred = best_Elastic.predict(X_test)

# חישוב ה-RMSE
mse = mean_squared_error(y_test, pred)
rmse = math.sqrt(mse)
print(f'RMSE: {rmse}')


Length of X_train: 1350
Length of X_test: 150
Length of y_train: 1350
Length of y_test: 150
Best alpha: 0.1
Best l1_ratio: 0.9
RMSE: 14296.06836155938


* 10-Fold Cross Validation

In [65]:

# שלב 1: המרת עמודות לקטגוריות
def convert_to_categorical(data, columns):
    for column in columns:
        data[column] = pd.Categorical(data[column]).codes
    return data

# עמודות שצריך להמיר לקטגוריות
categorical_columns = ['manufactor', 'Gear', 'Engine_type', 'Prev_ownership', 'Curr_ownership', 'Area', 'Color']

# המרת עמודות לקטגוריות
prepared_df_categorical = convert_to_categorical(prepared_df.copy(), categorical_columns)

# הגדרת המשתנים המסבירים והמשתנה המוסבר
X = prepared_df_categorical.drop('Price', axis=1)
y = prepared_df_categorical['Price']

# הגדרת המודל
model = ElasticNet()

# הגדרת KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# ביצוע חיזויים בעזרת cross_val_predict
predictions = cross_val_predict(model, X, y, cv=kf)

# יצירת DataFrame לתוצאות
correlations = pd.DataFrame(index=X.columns)

# חישוב הקורלציה בין כל עמודה לבין החיזויים
for column in X.columns:
    correlation = X[column].corr(pd.Series(predictions, index=X.index))
    correlations.loc[column, 'Correlation with Predictions'] = correlation

print("Correlation with predictions for all columns:")
print(correlations)


Correlation with predictions for all columns:
                 Correlation with Predictions
manufactor                          -0.169308
Year                                 0.872054
Hand                                -0.429846
Gear                                -0.179358
capacity_Engine                      0.073631
Engine_type                          0.103937
Prev_ownership                      -0.095033
Curr_ownership                      -0.049594
Area                                -0.007028
Color                                0.071165
Km                                  -0.713937


In [66]:
# חישוב ערך מוחלט של הקורלציות
correlations['Absolute Correlation'] = correlations['Correlation with Predictions'].abs()

# קבלת חמשת העמודות עם הקורלציות הרחוקות ביותר מהאפס
top_5_columns = correlations.nlargest(5, 'Absolute Correlation')

print("חמשת העמודות עם הקורלציות הרחוקות ביותר מהאפס:")
print(top_5_columns[['Correlation with Predictions']])

חמשת העמודות עם הקורלציות הרחוקות ביותר מהאפס:
            Correlation with Predictions
Year                            0.872054
Km                             -0.713937
Hand                           -0.429846
Gear                           -0.179358
manufactor                     -0.169308


In [67]:
correlations['Absolute Correlation'] = correlations['Correlation with Predictions'].abs()

# קבלת חמשת העמודות עם הקורלציות הרחוקות ביותר מהאפס
top_5_columns = correlations.nlargest(5, 'Absolute Correlation')

# הוספת עמודת "תיאור" עם חיובי/שלילי
top_5_columns['Sign'] = top_5_columns['Correlation with Predictions'].apply(lambda x: 'Positive' if x > 0 else 'Negative')

# הדפסת התוצאות
print("חמשת העמודות עם הקורלציות הרחוקות ביותר מהאפס:")
print(top_5_columns[['Correlation with Predictions', 'Sign']])

חמשת העמודות עם הקורלציות הרחוקות ביותר מהאפס:
            Correlation with Predictions      Sign
Year                            0.872054  Positive
Km                             -0.713937  Negative
Hand                           -0.429846  Negative
Gear                           -0.179358  Negative
manufactor                     -0.169308  Negative
